This notebook does performs the cleaning of New York City yellow taxi trip historical data. It starts by loading and merging several Parquet files with the selection of useful features, including pickup time, location IDs, trip distances, fares, and surcharges. The notebook deals with missing values, especially in such attributes as `passenger_count` and `congestion_surcharge` through the use of reasonable imputations. It also has validation processes like distribution checks to make sure that the data is consistent. The output dataset is organised and is used to continue analysis, modeling, and incorporation into the application.

In [1]:
# Importing.
import pandas as pd
import glob
import holidays
import matplotlib.pyplot as plt

In [2]:
# Disabling scientific notation.
pd.set_option('display.float_format', '{:.2f}'.format)

In [3]:
# File location.
files = glob.glob(r'C:\Users\marty\OneDrive\Desktop\Research Practicum\taxi_data\yellow_tripdata_*.parquet')

# Read and concatenate.
df = pd.concat([pd.read_parquet(file) for file in files])

# Saving combined file.
df.to_parquet(r'C:\Users\marty\OneDrive\Desktop\Research Practicum\taxi_data\combined_yellow_tripdata.parquet', index=False)

In [4]:
# Loading file.
df_combined = pd.read_parquet(r'C:\Users\marty\OneDrive\Desktop\Research Practicum\taxi_data\combined_yellow_tripdata.parquet')

# Previewing data.
print(df_combined.head())

# Display number of rows and columns.
print(df_combined.shape)

# See column names and data types.
print(df_combined.info())

# Summary statistics.
print(df_combined.describe())

   VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
0         2  2024-01-01 00:57:55   2024-01-01 01:17:43             1.00   
1         1  2024-01-01 00:03:00   2024-01-01 00:09:36             1.00   
2         1  2024-01-01 00:17:06   2024-01-01 00:35:01             1.00   
3         1  2024-01-01 00:36:38   2024-01-01 00:44:56             1.00   
4         1  2024-01-01 00:46:51   2024-01-01 00:52:57             1.00   

   trip_distance  RatecodeID store_and_fwd_flag  PULocationID  DOLocationID  \
0           1.72        1.00                  N           186            79   
1           1.80        1.00                  N           140           236   
2           4.70        1.00                  N           236            79   
3           1.40        1.00                  N            79           211   
4           0.80        1.00                  N           211           148   

   payment_type  fare_amount  extra  mta_tax  tip_amount  tolls_amount  \


In [5]:
# Loading parquet file.
df = df_combined.copy()

# Selecting useful features.
df = df[[
    "tpep_pickup_datetime",
    "tpep_dropoff_datetime",
    "PULocationID",
    "DOLocationID",
    "passenger_count",
    "trip_distance",
    "fare_amount",
    "tip_amount",
    "total_amount",
    "congestion_surcharge"
]]

# Convert datetime columns.
df["tpep_pickup_datetime"] = pd.to_datetime(df["tpep_pickup_datetime"])
df["tpep_dropoff_datetime"] = pd.to_datetime(df["tpep_dropoff_datetime"])

# New time-based features from pickup time.
df["pickup_hour"] = df["tpep_pickup_datetime"].dt.hour
df["pickup_dayofweek"] = df["tpep_pickup_datetime"].dt.dayofweek  # 0 is Monday, 6 is Sunday
df["is_weekend"] = df["pickup_dayofweek"] >= 5

# Get trip duration in minutes.
df["trip_duration_min"] = (
    (df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"]).dt.total_seconds() / 60).round(2)

# More readable version of trip duration.
df["trip_duration_readable"] = df["trip_duration_min"].apply(
    lambda x: f"{int(x):02d}:{int((x % 1) * 60):02d}")

# Display cleaned info.
print(df.head())
print(df.info())

  tpep_pickup_datetime tpep_dropoff_datetime  PULocationID  DOLocationID  \
0  2024-01-01 00:57:55   2024-01-01 01:17:43           186            79   
1  2024-01-01 00:03:00   2024-01-01 00:09:36           140           236   
2  2024-01-01 00:17:06   2024-01-01 00:35:01           236            79   
3  2024-01-01 00:36:38   2024-01-01 00:44:56            79           211   
4  2024-01-01 00:46:51   2024-01-01 00:52:57           211           148   

   passenger_count  trip_distance  fare_amount  tip_amount  total_amount  \
0             1.00           1.72        17.70        0.00         22.70   
1             1.00           1.80        10.00        3.75         18.75   
2             1.00           4.70        23.30        3.00         31.30   
3             1.00           1.40        10.00        2.00         17.00   
4             1.00           0.80         7.90        3.20         16.10   

   congestion_surcharge  pickup_hour  pickup_dayofweek  is_weekend  \
0               

In [6]:
# Check for NaN values.
print(df.isna().sum())

tpep_pickup_datetime            0
tpep_dropoff_datetime           0
PULocationID                    0
DOLocationID                    0
passenger_count           4091232
trip_distance                   0
fare_amount                     0
tip_amount                      0
total_amount                    0
congestion_surcharge      4091232
pickup_hour                     0
pickup_dayofweek                0
is_weekend                      0
trip_duration_min               0
trip_duration_readable          0
dtype: int64


In [7]:
# Examining the issue.
print(f"Total rows: {df.shape[0]}")
print(f"Total rows with missing passenger_count or congestion_surcharge: {df[df['passenger_count'].isna() | df['congestion_surcharge'].isna()].shape[0]}")

Total rows: 41169720
Total rows with missing passenger_count or congestion_surcharge: 4091232


In [8]:
# Let's examine the distribution.
print("Passenger Count Distribution:")
print(df["passenger_count"].value_counts(dropna=True, normalize=True).round(8) * 100)

print("Congestion Surcharge Distribution:")
print(df["congestion_surcharge"].value_counts(dropna=True, normalize=True).round(8) * 100)

Passenger Count Distribution:
passenger_count
1.00   77.22
2.00   14.59
3.00    3.46
4.00    2.20
0.00    1.08
5.00    0.86
6.00    0.58
8.00    0.00
7.00    0.00
9.00    0.00
Name: proportion, dtype: float64
Congestion Surcharge Distribution:
congestion_surcharge
2.50    90.62
0.00     8.04
-2.50    1.34
1.00     0.00
0.75     0.00
-0.75    0.00
2.00     0.00
2.52     0.00
Name: proportion, dtype: float64


In [9]:
# About 10% of rows are affected. However, passenger_count, and congestion_surcharge are not key features which is why I am imputing.
df["passenger_count"] = df["passenger_count"].fillna(1)

# Some are negative so set them to 0 if that's the case and impute with the mode as before.
df.loc[df["congestion_surcharge"] < 0, "congestion_surcharge"] = 0.0
df["congestion_surcharge"] = df["congestion_surcharge"].fillna(2.50)

In [10]:
# Check for NaN values again.
print(df.isna().sum())

tpep_pickup_datetime      0
tpep_dropoff_datetime     0
PULocationID              0
DOLocationID              0
passenger_count           0
trip_distance             0
fare_amount               0
tip_amount                0
total_amount              0
congestion_surcharge      0
pickup_hour               0
pickup_dayofweek          0
is_weekend                0
trip_duration_min         0
trip_duration_readable    0
dtype: int64


In [11]:
# These values should be greater than 0.
print("Zero/negative fare amounts:", (df["fare_amount"] <= 0).sum())
print("Zero/negative trip distance:", (df["trip_distance"] <= 0).sum())
print("Zero/negative duration:", (df["trip_duration_min"] <= 0).sum())

Zero/negative fare amounts: 748284
Zero/negative trip distance: 776305
Zero/negative duration: 13510


In [12]:
# Fare Amount.
fare_mode = df[df["fare_amount"] > 0]["fare_amount"].mode()[0]
fare_median = df[df["fare_amount"] > 0]["fare_amount"].median()

# Trip Duration.
duration_mode = df[df["trip_duration_min"] > 0]["trip_duration_min"].mode()[0]
duration_median = df[df["trip_duration_min"] > 0]["trip_duration_min"].median()

# Trip Distance.
distance_mode = df[df["trip_distance"] > 0]["trip_distance"].mode()[0]
distance_median = df[df["trip_distance"] > 0]["trip_distance"].median()

# Display mode and median for each.
print("Fare Amount:")
print(f"    Mode: {fare_mode}")
print(f"    Median: {fare_median}")

print("\nTrip Duration:")
print(f"    Mode: {duration_mode}")
print(f"    Median: {duration_median}")

print("\nTrip Distance:")
print(f"    Mode: {distance_mode}")
print(f"    Median: {distance_median}")

Fare Amount:
    Mode: 8.6
    Median: 14.2

Trip Duration:
    Mode: 9.0
    Median: 13.0

Trip Distance:
    Mode: 0.9
    Median: 1.8


In [13]:
# Getting medians from valid data.
fare_median = df[df["fare_amount"] > 0]["fare_amount"].median()
duration_median = df[df["trip_duration_min"] > 0]["trip_duration_min"].median()
distance_median = df[df["trip_distance"] > 0]["trip_distance"].median()

# Applying imputation.
df.loc[df["fare_amount"] <= 0, "fare_amount"] = fare_median
df.loc[df["trip_duration_min"] <= 0, "trip_duration_min"] = duration_median
df.loc[df["trip_distance"] <= 0, "trip_distance"] = distance_median

In [14]:
# Check again.
print("Zero/negative fare amounts:", (df["fare_amount"] <= 0).sum())
print("Zero/negative trip distance:", (df["trip_distance"] <= 0).sum())
print("Zero/negative duration:", (df["trip_duration_min"] <= 0).sum())

Zero/negative fare amounts: 0
Zero/negative trip distance: 0
Zero/negative duration: 0


In [15]:
# Check and discard duplicate rows.
duplicate_rows = df.duplicated().sum()
print(f"Duplicate rows: {duplicate_rows}")

df = df.drop_duplicates()

Duplicate rows: 4


In [16]:
# Calculating fare per mile and per minute.
df["fare_per_mile"] = df["fare_amount"] / df["trip_distance"]
df["fare_per_min"] = df["fare_amount"] / df["trip_duration_min"]

# Checking stats.
print("Fare per Mile Distribution:")
print(df["fare_per_mile"].describe())

print("\nFare per Minute Distribution:")
print(df["fare_per_min"].describe())

Fare per Mile Distribution:
count   41169716.00
mean          14.30
std          164.18
min            0.00
25%            5.75
50%            7.39
75%            9.53
max       185597.96
Name: fare_per_mile, dtype: float64

Fare per Minute Distribution:
count   41169716.00
mean           4.23
std           58.07
min            0.00
25%            0.98
50%            1.13
75%            1.37
max       112483.61
Name: fare_per_min, dtype: float64


In [17]:
# Setting realistic thresholds.
mile_threshold = 100
min_threshold = 20

# Counting them.
outliers_per_mile = (df["fare_per_mile"] >= mile_threshold).sum()
outliers_per_min = (df["fare_per_min"] >= min_threshold).sum()

# Results.
print(f"Outliers (fare_per_mile >= {mile_threshold}): {outliers_per_mile}")
print(f"Outliers (fare_per_min >= {min_threshold}): {outliers_per_min}")

Outliers (fare_per_mile >= 100): 190718
Outliers (fare_per_min >= 20): 360502


In [18]:
# Dropping rows as they are less than 1% of our data.
df = df[(df["fare_per_mile"] < mile_threshold) & (df["fare_per_min"] < min_threshold)]
print(f"Remaining rows after dropping invalid rows: {len(df):,}")

Remaining rows after dropping invalid rows: 40,715,193


In [19]:
# Check how many rows have pickup after dropoff.
invalid_time_rows_before = (df["tpep_pickup_datetime"] > df["tpep_dropoff_datetime"]).sum()
print(f"Before Dropping - Trips with pickup after dropoff: {invalid_time_rows_before}")

# Drop rows where pickup is after dropoff.
df = df[df["tpep_pickup_datetime"] <= df["tpep_dropoff_datetime"]]

# Check after dropping.
invalid_time_rows_after = (df["tpep_pickup_datetime"] > df["tpep_dropoff_datetime"]).sum()
print(f"After Dropping - Trips with pickup after dropoff: {invalid_time_rows_after}")

Before Dropping - Trips with pickup after dropoff: 1571
After Dropping - Trips with pickup after dropoff: 0


In [20]:
# Checking for rows where year is not 2024 (data should be only from 2024).
non_2024_pickups = df[df["tpep_pickup_datetime"].dt.year != 2024]
non_2024_dropoffs = df[df["tpep_dropoff_datetime"].dt.year != 2024]

print(f"Before Dropping - Rows with pickup year that's not 2024: {len(non_2024_pickups)}")
print(f"Before Dropping - Rows with dropoff year that's not 2024: {len(non_2024_dropoffs)}")

# Dropping rows not in 2024.
df = df[
    (df["tpep_pickup_datetime"].dt.year == 2024) &
    (df["tpep_dropoff_datetime"].dt.year == 2024)
]

# Checking after drop.
non_2024_pickups_after = df[df["tpep_pickup_datetime"].dt.year != 2024]
non_2024_dropoffs_after = df[df["tpep_dropoff_datetime"].dt.year != 2024]

print(f"After Dropping - Rows with pickup year that's not 2024: {len(non_2024_pickups_after)}")
print(f"After Dropping - Rows with dropoff year that's not 2024: {len(non_2024_dropoffs_after)}")

Before Dropping - Rows with pickup year that's not 2024: 56
Before Dropping - Rows with dropoff year that's not 2024: 659
After Dropping - Rows with pickup year that's not 2024: 0
After Dropping - Rows with dropoff year that's not 2024: 0


In [21]:
# Checking how many same zone short trips (< 2 mins) there are.
same_zone_short_trips = df[
    (df["PULocationID"] == df["DOLocationID"]) & (df["trip_duration_min"] < 2)
]
print(f"Same zone short trips to be dropped: {len(same_zone_short_trips)}")

# Dropping these rows as based on previous examination there are not that many considering the size of our data.
df = df[~(
    (df["PULocationID"] == df["DOLocationID"]) & 
    (df["trip_duration_min"] < 2)
)]

# Confirming if they were dropped.
remaining_same_zone_short_trips = df[
    (df["PULocationID"] == df["DOLocationID"]) & (df["trip_duration_min"] < 2)
]
print(f"Remaining same zone short trips: {len(remaining_same_zone_short_trips)}")

Same zone short trips to be dropped: 239556
Remaining same zone short trips: 0


In [22]:
# Below are Manhattan zones based on the documentation provided.
manhattan_zones = [4, 12, 13, 24, 41, 42, 43, 45, 48, 50, 68, 74, 75, 79, 87, 88, 90, 100, 103, 104, 105, 
107, 113, 114, 116, 120, 125, 127, 128, 137, 140, 141, 142, 143, 144, 148, 151, 152, 
153, 158, 161, 162, 163, 164, 166, 170, 186, 202, 209, 211, 224, 229, 230, 231, 232, 
233, 234, 236, 237, 238, 239, 243, 244, 246, 249, 261, 262, 263]

# Count rows where either pickup or dropoff is outside of Manhattan.
non_manhattan_rows = df[
    ~df["PULocationID"].isin(manhattan_zones) |
    ~df["DOLocationID"].isin(manhattan_zones)
]

# Display results.
print(f"Rows not fully within Manhattan: {len(non_manhattan_rows)}")

Rows not fully within Manhattan: 6782622


In [ ]:
# Keeping rows where both pickup and dropoff are in Manhattan.
df = df[
    df["PULocationID"].isin(manhattan_zones) &
    df["DOLocationID"].isin(manhattan_zones)
]

# Displaying results.
print(f"Remaining Manhattan only trips: {len(df):,}")

Remaining Manhattan only trips: 33,690,778


In [ ]:
# Dropping uneccessary columns.
df.reset_index(drop=True, inplace=True)
df.drop(columns=["fare_per_mile", "fare_per_min", "trip_speed_mph"], inplace=True, errors="ignore")

In [25]:
# Examining cleaned data.
print(df.head())

  tpep_pickup_datetime tpep_dropoff_datetime  PULocationID  DOLocationID  \
0  2024-01-01 00:57:55   2024-01-01 01:17:43           186            79   
1  2024-01-01 00:03:00   2024-01-01 00:09:36           140           236   
2  2024-01-01 00:17:06   2024-01-01 00:35:01           236            79   
3  2024-01-01 00:36:38   2024-01-01 00:44:56            79           211   
4  2024-01-01 00:46:51   2024-01-01 00:52:57           211           148   

   passenger_count  trip_distance  fare_amount  tip_amount  total_amount  \
0             1.00           1.72        17.70        0.00         22.70   
1             1.00           1.80        10.00        3.75         18.75   
2             1.00           4.70        23.30        3.00         31.30   
3             1.00           1.40        10.00        2.00         17.00   
4             1.00           0.80         7.90        3.20         16.10   

   congestion_surcharge  pickup_hour  pickup_dayofweek  is_weekend  \
0               

In [26]:
# Additional features for ML.
df["pickup_date"] = df["tpep_pickup_datetime"].dt.date
df["pickup_month"] = df["tpep_pickup_datetime"].dt.month
df["pickup_weekofyear"] = df["tpep_pickup_datetime"].dt.isocalendar().week
df["pickup_minute"] = df["tpep_pickup_datetime"].dt.minute

In [27]:
# Feature to check if date is a holiday.
us_holidays = holidays.US(years=2024)
df["is_holiday"] = df["tpep_pickup_datetime"].dt.date.isin(us_holidays)

In [28]:
# Rounding pickup time to the nearest hour.
df['pickup_hour_ts'] = df['tpep_pickup_datetime'].dt.floor('h')

# Counting trips for each zone and hour combination.
zone_hourly_counts = (
    df.groupby(['PULocationID', 'pickup_hour_ts'])
    .size()
    .reset_index(name='zone_hourly_pickups')
)

# Merging counts.
df = df.merge(zone_hourly_counts, on=['PULocationID', 'pickup_hour_ts'], how='left')

# Examining result.
columns_to_show = [col for col in ['tpep_pickup_datetime', 'PULocationID', 'pickup_hour_ts', 'zone_hourly_pickups'] if col in df.columns]
print(df[columns_to_show].head())

  tpep_pickup_datetime  PULocationID pickup_hour_ts  zone_hourly_pickups
0  2024-01-01 00:57:55           186     2024-01-01                   88
1  2024-01-01 00:03:00           140     2024-01-01                   91
2  2024-01-01 00:17:06           236     2024-01-01                  160
3  2024-01-01 00:36:38            79     2024-01-01                  328
4  2024-01-01 00:46:51           211     2024-01-01                   49


In [ ]:
# Rounding dropoff time to the nearest hour.
df['dropoff_hour_ts'] = df['tpep_dropoff_datetime'].dt.floor('h')

# Counting dropoffs for each zone and hour combination.
zone_hourly_dropoffs = (
    df.groupby(['DOLocationID', 'dropoff_hour_ts'])
    .size()
    .reset_index(name='zone_hourly_dropoffs')
)

# Merging dropoff counts.
df = df.merge(zone_hourly_dropoffs, on=['DOLocationID', 'dropoff_hour_ts'], how='left')

# Examining results.
columns_to_show = [col for col in ['tpep_dropoff_datetime', 'DOLocationID', 'dropoff_hour_ts', 'zone_hourly_dropoffs'] if col in df.columns]
print(df[columns_to_show].head())

  tpep_dropoff_datetime  DOLocationID     dropoff_hour_ts  \
0   2024-01-01 01:17:43            79 2024-01-01 01:00:00   
1   2024-01-01 00:09:36           236 2024-01-01 00:00:00   
2   2024-01-01 00:35:01            79 2024-01-01 00:00:00   
3   2024-01-01 00:44:56           211 2024-01-01 00:00:00   
4   2024-01-01 00:52:57           148 2024-01-01 00:00:00   

   zone_hourly_dropoffs  
0                   281  
1                   169  
2                   183  
3                    35  
4                    60  


In [30]:
# Adding a total activity feature that counts pickups and dropoffs.
df['zone_hourly_total_activity'] = df['zone_hourly_pickups'] + df['zone_hourly_dropoffs']

In [ ]:
# Examining data.
print(df.head())

  tpep_pickup_datetime tpep_dropoff_datetime  PULocationID  DOLocationID  \
0  2024-01-01 00:57:55   2024-01-01 01:17:43           186            79   
1  2024-01-01 00:03:00   2024-01-01 00:09:36           140           236   
2  2024-01-01 00:17:06   2024-01-01 00:35:01           236            79   
3  2024-01-01 00:36:38   2024-01-01 00:44:56            79           211   
4  2024-01-01 00:46:51   2024-01-01 00:52:57           211           148   

   passenger_count  trip_distance  fare_amount  tip_amount  total_amount  \
0             1.00           1.72        17.70        0.00         22.70   
1             1.00           1.80        10.00        3.75         18.75   
2             1.00           4.70        23.30        3.00         31.30   
3             1.00           1.40        10.00        2.00         17.00   
4             1.00           0.80         7.90        3.20         16.10   

   congestion_surcharge  ...  pickup_date  pickup_month  pickup_weekofyear  \
0       

In [32]:
# Adding a feature that shows which part of the day it is.
def assign_day_part(hour):
    if 5 <= hour < 12:
        return "morning"
    elif 12 <= hour < 17:
        return "afternoon"
    elif 17 <= hour < 21:
        return "evening"
    else:
        return "night"

df["day_part"] = df["pickup_hour"].apply(assign_day_part)

In [33]:
# Adding expected busy (peak) hour feature.
df["is_peak_hour"] = df["pickup_hour"].isin([7, 8, 9, 16, 17, 18])

In [34]:
# Adding a season feature.
def assign_season(month):
    if month in [12, 1, 2]:
        return "winter"
    elif month in [3, 4, 5]:
        return "spring"
    elif month in [6, 7, 8]:
        return "summer"
    else:
        return "fall"

df["season"] = df["pickup_month"].apply(assign_season)

In [ ]:
# Dropping unnecessary columns.
df = df.drop(columns=[
    'fare_amount',
    'tip_amount',
    'total_amount',
    'congestion_surcharge', # After examination congestion_sur

], errors='ignore')

In [36]:
# Final data for modeling.
df.to_parquet("manhattan_taxi_2024_clean.parquet", index=False)

# Preview which can be viewed using excel.
df.sample(10000).to_csv("manhattan_taxi_2024_sample.csv", index=False)